In [1]:
import pandas as pd
import pickle
import catboost as ctb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('./data/df_train.csv')


In [3]:
df_train['Year']  = df_train['time'].astype('datetime64').dt.year
df_train['Month_of_year']  = df_train['time'].astype('datetime64').dt.month
df_train['Week_of_year'] = df_train['time'].astype('datetime64').dt.weekofyear
df_train['Day_of_year']  = df_train['time'].astype('datetime64').dt.dayofyear
df_train['Day_of_month']  = df_train['time'].astype('datetime64').dt.day
df_train['Day_of_week'] = df_train['time'].astype('datetime64').dt.dayofweek
df_train['Hour_of_week'] = ((df_train['time'].astype('datetime64').dt.dayofweek) * 24 + 24) - (24 - df_train['time'].astype('datetime64').dt.hour)

df_train = df_train.drop(columns=['Week_of_year','Day_of_year','Hour_of_week', 'Unnamed: 0','time'])  

In [4]:
df_train['Valencia_wind_deg'] = df_train['Valencia_wind_deg'].str.extract('(\d+)').astype('int64')

    # change the test data type to integer
df_train['Valencia_wind_deg'] = pd.to_numeric(df_train['Valencia_wind_deg'])

df_train['Valencia_pressure'] = df_train['Valencia_pressure'].fillna(df_train['Valencia_pressure'].mode()[0])
df_train['Seville_pressure'] = df_train['Seville_pressure'].str.extract('(\d+)').astype('int64')
 
    # change the data type to integer
df_train['Seville_pressure'] = pd.to_numeric(df_train['Seville_pressure'])

In [6]:
y_train = df_train[['load_shortfall_3h']]
X_train = df_train[:len(df_train)].drop('load_shortfall_3h', axis=1)

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_scaled = pd.DataFrame(X_scaled,columns=X_train.columns)

In [8]:
model_CBR = ctb.CatBoostRegressor()
model_CBR.fit(X_train, y_train)

Learning rate set to 0.057692
0:	learn: 5133.0776953	total: 203ms	remaining: 3m 22s
1:	learn: 5048.5782361	total: 243ms	remaining: 2m 1s
2:	learn: 4970.9696008	total: 277ms	remaining: 1m 31s
3:	learn: 4907.6770418	total: 311ms	remaining: 1m 17s
4:	learn: 4851.7836358	total: 339ms	remaining: 1m 7s
5:	learn: 4794.1742389	total: 366ms	remaining: 1m
6:	learn: 4733.7313255	total: 397ms	remaining: 56.4s
7:	learn: 4693.3257158	total: 443ms	remaining: 55s
8:	learn: 4640.7018141	total: 472ms	remaining: 52s
9:	learn: 4603.5934076	total: 499ms	remaining: 49.4s
10:	learn: 4562.5300613	total: 529ms	remaining: 47.6s
11:	learn: 4528.5912978	total: 560ms	remaining: 46.1s
12:	learn: 4489.1585135	total: 590ms	remaining: 44.8s
13:	learn: 4451.8556144	total: 617ms	remaining: 43.5s
14:	learn: 4417.8347415	total: 663ms	remaining: 43.5s
15:	learn: 4388.0013072	total: 692ms	remaining: 42.6s
16:	learn: 4359.2852371	total: 721ms	remaining: 41.7s
17:	learn: 4336.9388318	total: 755ms	remaining: 41.2s
18:	learn: 4

In [9]:
# Pickle model for use within our API
save_path = '../assets/trained-models/model_CB.pkl'
print (f"Training completed. Saving model to: {save_path}")
pickle.dump(model_CBR, open(save_path,'wb'))

Training completed. Saving model to: ../assets/trained-models/model_CB.pkl
